# REQUEST FILE GENERATION

In [1]:
import random
import networkx as nx

def generate_requests(graph, num_requests, demand_type):
    result = []
    node_request = len(graph.nodes) 
    
    for _ in range(num_requests):
        source = random.choice(list(graph.nodes))
        destination = random.choice(list(graph.nodes))
        while destination == source:
            destination = random.choice(list(graph.nodes))
        if demand_type == 'variable':
            demand = 12.5 * random.choice([1, 2, 3, 4, 5, 6])
        else:
            demand = demand
        
        new_req = {
            "source": source,
            "destination": destination,
            "demand": demand
        }
        
        result.append(new_req)
        
    return result

if __name__ == "__main__":
    file_path = input("Enter file name : ")
    
    try:
        graph = nx.read_gpickle(file_path+".gpickle")
        num_requests = int(input("Enter the number of requests: "))
        demand_type = input("Enter demand type (40 or 80 for fixed, or variable): ").lower()
        if demand_type == '40':
            demand = 40
            
            
        elif demand_type == '80':
            demand = 80
        elif demand_type == 'variable':
            demand = "variable"
        else:
            print("Invalid demand type.")
            exit()
        requests = generate_requests(graph, num_requests, demand_type)
        for req in requests:
            print(req)
    
    except FileNotFoundError:
        print("The specified gpickle file was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")


Enter file name : network_1
Enter the number of requests: 100
Enter demand type (40 or 80 for fixed, or variable): variable
{'source': 13, 'destination': 19, 'demand': 37.5}
{'source': 7, 'destination': 13, 'demand': 50.0}
{'source': 5, 'destination': 18, 'demand': 37.5}
{'source': 4, 'destination': 5, 'demand': 25.0}
{'source': 13, 'destination': 5, 'demand': 62.5}
{'source': 13, 'destination': 21, 'demand': 12.5}
{'source': 18, 'destination': 2, 'demand': 12.5}
{'source': 16, 'destination': 3, 'demand': 50.0}
{'source': 5, 'destination': 4, 'demand': 62.5}
{'source': 15, 'destination': 22, 'demand': 75.0}
{'source': 11, 'destination': 4, 'demand': 75.0}
{'source': 10, 'destination': 12, 'demand': 75.0}
{'source': 1, 'destination': 17, 'demand': 62.5}
{'source': 18, 'destination': 22, 'demand': 25.0}
{'source': 15, 'destination': 21, 'demand': 62.5}
{'source': 17, 'destination': 6, 'demand': 25.0}
{'source': 10, 'destination': 15, 'demand': 62.5}
{'source': 19, 'destination': 11, 'dem

# DATA GENERATION

In [2]:
import networkx as nx

def load_network_graph(filename):
    return nx.read_gpickle(filename)

def sparse_regeneration_algorithm(network_graph, requests, distance_threshold , Frequency_Slot_Capacity ):
    results = [] 
    Total_Regen=0
    Total_Frequency_Slot=0
    Guard_Frequency_Slot=2
    for request in requests:
        source = request['source']
        destination = request['destination']
        demand = request['demand']  
        
        
        try:   
            primary_path = nx.dijkstra_path(graph, source=source, target=destination, weight='weight')
            total_distance_primary = sum(graph[primary_path[i]][primary_path[i+1]]['weight'] for i in range(len(primary_path) - 1))
            
            graph_backup = graph.copy()
            for i in range(len(primary_path) - 1):
                graph_backup.remove_edge(primary_path[i], primary_path[i+1])
            try:
                backup_path = nx.dijkstra_path(graph_backup, source=source, target=destination, weight='weight')
                total_distance_backup = sum(graph_backup[backup_path[i]][backup_path[i+1]]['weight'] for i in range(len(backup_path) - 1))
            except nx.NetworkXNoPath:
                backup_path = None
                total_distance_backup = None
            
            regenerations = 0
            current_distance = 0
            regeneration_nodes = []
            
            
            for i in range(1, len(primary_path)):
                link_distance = graph[primary_path[i-1]][primary_path[i]]['weight']
                current_distance += link_distance 
                N = (demand/Frequency_Slot_Capacity)+Guard_Frequency_Slot
                Total_Frequency_Slot+=N
                
                
                if current_distance > distance_threshold:
                    regenerations += 1
                    Total_Regen+=1
                    regeneration_nodes.append(primary_path[i-1])
                    current_distance -= distance_threshold
            
            results.append({
                'source': source,
                'destination': destination,
                'demand':demand,
                'primary_path': primary_path,
                'total_distance_primary': total_distance_primary,
                'backup_path':backup_path,
                'total_distance_backup': total_distance_backup,
                'regenerations': regenerations,
                'regeneration_nodes': regeneration_nodes,
                'Total_Regenerations' : Total_Regen,
                'Total_Frequency_Slot' : Total_Frequency_Slot,
                'Spectrum_Allocated':N,
                
            })
        except nx.NetworkXNoPath:
            print(f"No path found between {source} and {destination}.")
            results.append({
                'source': source,
                'destination': destination,
                'demand':None,
                'primary_path': None,
                'total_distance_primary': None,
                'backup_path':None,
                'total_distance_backup': None,
                'regenerations': None,
                'regeneration_nodes': None,
                'Total_Regenerations' : None,
                'Total_Frequency_slot' : None,
                'Spectrum_Allocated':None,
            })
    return results

network_graph=graph

distance_threshold = 2000  
Frequency_Slot_Capacity = 25

results = sparse_regeneration_algorithm(network_graph, requests, distance_threshold , Frequency_Slot_Capacity)

for result in results:
    print("\nRequest from {} to {}".format(result['source'], result['destination']))
    print("  Demand:",result["demand"])
    print("  Path (primary):", result['primary_path'])
    print("  Backup Path:", result['backup_path'])
    print("  Total Distance (primary):", result['total_distance_primary'])
    print("  Total Distance (Backup):", result['total_distance_backup'])
    print("  Regenerations Required:", result['regenerations'])
    print("  Regeneration_at_nodes:",result["regeneration_nodes"])
    print("  Spectrum Allocated:",result["Spectrum_Allocated"])
print("\n\nTotal_Regenerations:" ,result["Total_Regenerations"])
print("Total_Frequency_Slot:" ,result["Total_Frequency_Slot"])


Request from 13 to 19
  Demand: 37.5
  Path (primary): [13, 12, 16, 21, 20, 19]
  Backup Path: [13, 17, 16, 15, 14, 19]
  Total Distance (primary): 3600
  Total Distance (Backup): 4900
  Regenerations Required: 1
  Regeneration_at_nodes: [16]
  Spectrum Allocated: 3.5

Request from 7 to 13
  Demand: 50.0
  Path (primary): [7, 9, 13]
  Backup Path: [7, 6, 8, 11, 12, 13]
  Total Distance (primary): 1750
  Total Distance (Backup): 4700
  Regenerations Required: 0
  Regeneration_at_nodes: []
  Spectrum Allocated: 4.0

Request from 5 to 18
  Demand: 37.5
  Path (primary): [5, 10, 18]
  Backup Path: [5, 8, 11, 15, 20, 19, 18]
  Total Distance (primary): 4500
  Total Distance (Backup): 6100
  Regenerations Required: 1
  Regeneration_at_nodes: [10]
  Spectrum Allocated: 3.5

Request from 4 to 5
  Demand: 25.0
  Path (primary): [4, 3, 6, 5]
  Backup Path: [4, 2, 1, 5]
  Total Distance (primary): 2650
  Total Distance (Backup): 2850
  Regenerations Required: 1
  Regeneration_at_nodes: [6]
  Spe